In [18]:
# input = """##########
# #..O..O.O#
# #......O.#
# #.OO..O.O#
# #..O@..O.#
# #O#..O...#
# #O..O..O.#
# #.OO.O.OO#
# #....O...#
# ##########

# <vv>^<v^>v>^vv^v>v<>v^v<v<^vv<<<^><<><>>v<vvv<>^v^>^<<<><<v<<<v^vv^v>^
# vvv<<^>^v^^><<>>><>^<<><^vv^^<>vvv<>><^^v>^>vv<>v<<<<v<^v>^<^^>>>^<v<v
# ><>vv>v^v^<>><>>>><^^>vv>v<^^^>>v^v^<^^>v^^>v^<^v>v<>>v^v^<v>v^^<^^vv<
# <<v<^>>^^^^>>>v^<>vvv^><v<<<>^^^vv^<vvv>^>v<^^^^v<>^>vvvv><>>v^<<^^^^^
# ^><^><>>><>^^<<^^v>>><^<v>^<vv>>v>>>^v><>^v><<<<v>>v<v<v>vvv>^<><<>^><
# ^>><>^v<><^vvv<^^<><v<<<<<><^v<<<><<<^^<v<^^^><^>>^<v^><<<^>>^v<v^v<v^
# >^>>^v>vv>^<<^v<>><<><<v<<v><>v<^vv<<<>^^v^>^^>>><<^v>>v^v><^^>>^<>vv^
# <><^^>^^^<><vvvvv^v<v<<>^v<v>v<<^><<><<><<<^^<<<^<<>><<><^^^>^^<>^>v<>
# ^^>vv<^v^v<vv>^<><v<^v>^^^>>>^^vvv^>vvv<>>>^<^>>>>>^<<^v>^vvv<>^<><<v>
# v^^>>><<^^<>>^v^<v^vv<>v^<<>^<^v^v><^<<<><<^<v><v<>vv>>v><v^<vv<>v^<<^"""

input = open("inputs/15").read()

In [19]:
import numpy as np


def parse_board(input):
    return np.array(list(map(list, input.splitlines())))


def add_tuple(t1, t2):
    return tuple(x + y for x, y in zip(t1, t2))


def print_board(board):
    print("\n".join("".join(row) for row in board))


dirs = {
    "^": (-1, 0),
    ">": (0, 1),
    "v": (1, 0),
    "<": (0, -1),
}

In [20]:
board_str, movement_str = input.split("\n\n")
movement = movement_str.replace("\n", "")

In [21]:
# we could elide the boundary of "#" but it's actually convenient to have it
board = parse_board(board_str)
board

array([['#', '#', '#', ..., '#', '#', '#'],
       ['#', '.', '.', ..., '.', '.', '#'],
       ['#', '#', '.', ..., 'O', 'O', '#'],
       ...,
       ['#', '.', '.', ..., '.', '.', '#'],
       ['#', 'O', '.', ..., '#', '.', '#'],
       ['#', '#', '#', ..., '#', '#', '#']], dtype='<U1')

In [22]:
def follow_box_chain(board_state, start, dir):
    """
    Follow the box chain until we hit a non-box character. Return the position of the last box and the character at that position.
    """
    assert board_state[start] == "O"

    pos = start
    # this loop guaranteed to terminate because the board is bordered by "#"
    while True:
        pos = add_tuple(pos, dir)
        if board_state[pos] == "O":
            pass
        else:
            return pos, board_state[pos]

In [23]:
start = list(zip(*np.where(board == "@")))[0]
board_state = board.copy()
board_state[start] = "."

current_pos = start
for move in movement:
    next_pos = add_tuple(current_pos, dirs[move])
    next_char = board_state[next_pos]

    if next_char == ".":
        current_pos = next_pos
    elif next_char == "O":
        non_box_pos, non_box_char = follow_box_chain(board_state, next_pos, dirs[move])

        if non_box_char == "#":
            # we can't move any further: the box chain is against a wall
            pass
        elif non_box_char == ".":
            # the box chain moves down, and the next pos frees up, and we move into it
            board_state[non_box_pos] = "O"
            board_state[next_pos] = "."
            current_pos = next_pos
        else:
            raise ValueError(f"Invalid character {non_box_char} at {non_box_pos}")
    elif next_char == "#":
        # we can't move any further: we're against a wall
        pass
    else:
        raise ValueError(f"Invalid character {next_char} at {next_pos}")

In [24]:
board_state

array([['#', '#', '#', ..., '#', '#', '#'],
       ['#', '.', '.', ..., 'O', 'O', '#'],
       ['#', '#', 'O', ..., 'O', 'O', '#'],
       ...,
       ['#', 'O', 'O', ..., 'O', 'O', '#'],
       ['#', 'O', 'O', ..., '#', '.', '#'],
       ['#', '#', '#', ..., '#', '#', '#']], dtype='<U1')

In [25]:
sm = 0
for i, pos in enumerate(zip(*np.where(board_state == "O"))):
    sm += 100 * pos[0] + pos[1]
sm

np.int64(1505963)

In [26]:
def transform_warehouse_map(original_map):
    transform_dict = {
        "#": ["#", "#"],
        "O": ["[", "]"],
        ".": [".", "."],
        "@": ["@", "."],
    }

    height, width = original_map.shape
    new_map = np.empty((height, width * 2), dtype=str)

    for old_char, new_chars in transform_dict.items():
        mask = original_map == old_char
        rows, cols = np.where(mask)
        new_map[rows, cols * 2] = new_chars[0]
        new_map[rows, cols * 2 + 1] = new_chars[1]

    return new_map


In [27]:
board = transform_warehouse_map(board)
start = list(zip(*np.where(board == "@")))[0]
board_state = board.copy()
board_state[start] = "."
current_pos = start

In [28]:
print_board(board_state)

####################################################################################################
##....##....[][][]......[]..[][]....##........[]..##..........[]..####..................[]........##
####....[][]..[][]..[]....####......[]........[]..[]##..[]....[]..##........[]....[]##......[][][]##
##[]....[]..[]........##........[]..[]......[]..........##..##..[]........##[][]####..[]..[]..[]..##
##[]..[]......[]..[][]..[]..##[][]......[]......[]....[]..##[][]..[]....[]........[][]......[][][]##
##[]..##..[][]..[][]..[]....[]......[][][]....##..[]..[]##[]..##............##[]......##..........##
##......##[]......[]..[]..............[]..[][]......[]........[]....[][]..[]........[]..[][]....[]##
##[]##..[]..[][][]......[]##[]..................[][]......[][][]......##..............##....##....##
####..[]............[]....[]......[]..##................[]......[]....[]............[][]..[][][][]##
##......[]....[]................[]....[]..[]..[]..............................[]......##..[

In [29]:
print_board(board_state)

####################################################################################################
##....##....[][][]......[]..[][]....##........[]..##..........[]..####..................[]........##
####....[][]..[][]..[]....####......[]........[]..[]##..[]....[]..##........[]....[]##......[][][]##
##[]....[]..[]........##........[]..[]......[]..........##..##..[]........##[][]####..[]..[]..[]..##
##[]..[]......[]..[][]..[]..##[][]......[]......[]....[]..##[][]..[]....[]........[][]......[][][]##
##[]..##..[][]..[][]..[]....[]......[][][]....##..[]..[]##[]..##............##[]......##..........##
##......##[]......[]..[]..............[]..[][]......[]........[]....[][]..[]........[]..[][]....[]##
##[]##..[]..[][][]......[]##[]..................[][]......[][][]......##..............##....##....##
####..[]............[]....[]......[]..##................[]......[]....[]............[][]..[][][][]##
##......[]....[]................[]....[]..[]..[]..............................[]......##..[

In [30]:
board_state[start]

np.str_('.')

In [31]:
start

(np.int64(24), np.int64(48))

In [32]:
def follow_box_chain_pt2(board_state, start, dir):
    visited = set()

    def follow_box_chain_pt2_recurse(board_state, start, dir):
        if start in visited:
            return True

        visited.add(start)

        current_char = board_state[start]
        if not ((current_char == "[") or (current_char == "]")):
            raise ValueError("Expected a box here")

        side_ok = follow_box_chain_pt2_recurse(
            board_state,
            add_tuple(start, dirs[">" if current_char == "[" else "<"]),
            dir,
        )

        # short circuit
        if not side_ok:
            return False

        next_pos = add_tuple(start, dirs[dir])
        next_char = board_state[next_pos]

        if next_char == "#":
            # we can't move any further: we're against a wall
            return False
        elif next_char == ".":
            # open space: ok
            return True
        elif next_char == "[" or next_char == "]":
            return follow_box_chain_pt2_recurse(board_state, next_pos, dir)
        else:
            raise ValueError(f"Invalid character {next_char} at {next_pos}")

    feasible = follow_box_chain_pt2_recurse(board_state, start, dir)
    return visited if feasible else None


for move in movement:
    next_pos = add_tuple(current_pos, dirs[move])
    next_char = board_state[next_pos]

    if next_char == ".":
        print("simple move")
        current_pos = next_pos
    elif next_char == "[" or next_char == "]":
        visited = follow_box_chain_pt2(board_state, next_pos, move)

        if visited is not None:
            # boxes are going to move. so let's store the values which are going to move, set them to '.', then "paste" the values one step in dir
            pos_to_change = list(visited)
            x, y = map(np.array, zip(*pos_to_change))
            new_x = x + dirs[move][0]
            new_y = y + dirs[move][1]

            old_values = board_state[x, y]
            board_state[x, y] = "."
            board_state[new_x, new_y] = old_values

            current_pos = next_pos
        else:
            print("move not feasible")

    elif next_char == "#":
        # we can't move any further: we're against a wall
        print("wall")
        pass
    else:
        raise ValueError(f"Invalid character {next_char} at {next_pos}")

wall
simple move
wall
wall
wall
simple move
simple move
simple move
simple move
wall
simple move
simple move
simple move
simple move
simple move
simple move
simple move
simple move
simple move
simple move
simple move
simple move
simple move
move not feasible
simple move
simple move
move not feasible
simple move
simple move
simple move
simple move
simple move
move not feasible
simple move
simple move
simple move
simple move
simple move
move not feasible
simple move
simple move
simple move
simple move
simple move
simple move
simple move
simple move
simple move
simple move
simple move
simple move
simple move
simple move
simple move
simple move
simple move
simple move
simple move
simple move
simple move
simple move
simple move
simple move
simple move
simple move
simple move
move not feasible
simple move
simple move
simple move
simple move
simple move
simple move
simple move
simple move
simple move
simple move
simple move
simple move
move not feasible
move not feasible
simple move
simple mo

In [33]:
print_board(board_state)

####################################################################################################
##....##....[][][]......[]..[][]....##........[]..##[]........[][]####[].......[].......[][][][]..##
####....[][]..[][]..[]....####......[]........[]..[]##....[]....[]##[].........[].[]##..[][][]..[]##
##[]....[]..[]........##........[]..[]......[]....[]....##[]##[]..........##....####.[].........[]##
##[]..[]......[]..[][]..[]..##[][]......[]....[][][]..[]..##[][]............[]....[][]........[][]##
##[]..##..[][]..[][]..[]....[]......[][][]....##..[]..[]##[][]##............##[]..[][]##....[]....##
##......##[]......[]..[]..............[]..[][]....[]........[]..............[]....[][]....[]..[][]##
##[]##..[]..[][][]......[]##[]..................[][]..................##....[]....[]..##..[]##....##
####..[]............[]....[]......[]..##.....................[].......[].......[][][][].....[][][]##
##......[]....[]................[]....[]..[]..[]...[].............................[][]##..[

In [34]:
sm = 0
for i, j in zip(*np.where(board_state == "[")):
    sm += 100 * i + j
sm

np.int64(1543141)